# 1. Install Requirements

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.8 MB/s eta 0:00:00


# 2. Load LaBSE

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings = model.encode(sentences)
print(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

[[ 0.02882476 -0.00602381 -0.05947006 ... -0.0300225  -0.02960703
   0.00067479]
 [-0.05550232  0.02546487 -0.02157256 ...  0.02932104  0.01150042
  -0.00848786]]


# 3. Load & Preprocess Data

To run cell below you should put all data files on the same folder as this notebook.

In [ ]:
import pandas as pd
import json

# with open("train.model-agnostic.json", "r") as f:
#     train_agnostic = pd.DataFrame(json.loads(f.read()))
# with open("train.model-aware.json", "r") as f:
#     train_aware = pd.DataFrame(json.loads(f.read()))
# with open("val.model-agnostic.json", "r") as f:
#     val_agnostic = pd.DataFrame(json.loads(f.read()))
# with open("val.model-aware.json", "r") as f:
#     val_aware = pd.DataFrame(json.loads(f.read()))
# with open("trial-v1.json", "r") as f:
#     test = pd.DataFrame(json.loads(f.read()))
with open("test.model-agnostic.json", "r") as f:
    agnostic = pd.DataFrame(json.loads(f.read()))
with open("test.model-aware.json", "r") as f:
    aware = pd.DataFrame(json.loads(f.read()))

> "The ref key therefore indicate whether target, source or both of these fields contain the semantic information necessary to establish whether a datapoint is a hallucination."

In [ ]:
tasks = ['MT', 'DM', 'PG']

def preprocess_data(data_df):
    output = {"hyp": {'all': [], 'MT': [], 'DM': [], 'PG': []},
              "tgt": {'all': [], 'MT': [], 'DM': [], 'PG': []},
              "labels": {'all': [], 'MT': [], 'DM': [], 'PG': []}}
    for index, row in data_df.iterrows():
        hyp = row['hyp']
        if row['ref'] == 'either' or row['ref'] == 'tgt':
            tgt = row['tgt']
        else:
            tgt = row['src']

        task = row['task'].strip()
        if task in tasks:
            output['hyp'][task].append(hyp)
            output['tgt'][task].append(tgt)
            output['labels'][task].append(row['p(Hallucination)'])

            output['hyp']['all'].append(hyp)
            output['tgt']['all'].append(tgt)
            output['labels']['all'].append(row['p(Hallucination)'])

    return output

In [ ]:
# p_val_agnostic = preprocess_data(val_agnostic)
# p_val_aware = preprocess_data(val_aware)
# p_test = preprocess_data(test)
# agnostic = preprocess_data(agnostic)
# aware = preprocess_data(aware)

KeyError: 'ref'

In [ ]:
import numpy as np
from tqdm import tqdm
output = []
for row in tqdm(aware.iterrows()):
    embedding_hyp = model.encode(row[1].hyp)
    if row[1].task == "PG":
        embedding_tgt = model.encode(row[1].src)
    elif row[1].task == "MT" or row[1].task == "DM":
        embedding_tgt = model.encode(row[1].tgt)


    similarity = np.sum(np.multiply(embedding_hyp, embedding_tgt))
    if similarity > 1:
        print(similarity)
        similarity=1
    elif similarity < 0:
        print(similarity)
        similarity = 0


    output.append({
        'id': row[1].id,
        'label': 'Hallucination' if similarity <= 0.5 else 'Not Hallucination',
        'p(Hallucination)': 1 - similarity
    })


output_path = "./test.model-aware_lapse.json"
f1 = open(output_path, 'w', encoding='utf-8')
json.dump(output, f1)
f1.close()

497it [00:15, 35.05it/s]

-0.016147695


541it [00:16, 36.25it/s]

-0.016093722


1500it [00:46, 32.45it/s]


In [ ]:
agnostic

,id,src,tgt,hyp,task
0,1,"Ты удивишься, если я скажу, что на самом деле ...",Would you be surprised if I told you my name i...,You're gonna be surprised if I say my real nam...,MT
1,2,Еды будет полно.,There will be plenty of food.,The food will be full.,MT
2,3,"Думаете, Том будет меня ждать?",Do you think that Tom will wait for me?,You think Tom's gonna wait for me?,MT
3,6,Два брата довольно разные.,The two brothers are pretty different.,There's a lot of friends.,MT
4,7,<define> Infradiaphragmatic </define> intra- a...,(medicine) Below the diaphragm.,(anatomy) Relating to the diaphragm.,DM
...,...,...,...,...,...
1495,2992,"Я никогда не говорил Мэри, что чувствую.",I never told Mary how I felt.,I've never told Mary I feel.,MT
1496,2993,""" Can he beat this rat - tailed <define> kyood...","A mutt, dog of mixed breed of little value; no...",(slang) A mustang.,DM
1497,2994,"Ты знаешь, почему они прекратили говорить?",Do you know why they stopped talking?,You know why they stopped talking?,MT
1498,2996,Can anyone back you up on that?,Can anyone confirm that?,Can anyone corroborate that?,PG


# 4. Get Embeddings & Calculate Similarity Scores

In [ ]:
import numpy as np

def encode_LaBSE(p_data, model):
    output = {"hyp": {},
              "tgt": {},
              "labels": {}}
    for key, value in p_data['hyp'].items():
        output['hyp'][key] = model.encode(value)
    for key, value in p_data['tgt'].items():
        output['tgt'][key] = model.encode(value)
    for key, value in p_data['labels'].items():
        output['labels'][key] = np.array(value)

    return output

In [ ]:
e_val_agnostic = encode_LaBSE(p_val_agnostic, model)
e_val_aware = encode_LaBSE(p_val_aware, model)
e_test = encode_LaBSE(p_test, model)

In [ ]:
e_test['hyp']['all'].shape

In [ ]:
def similarity(embeddings_1, embeddings_2):
    return np.sum(np.multiply(embeddings_1, embeddings_2), 1)

def calculate_similarity(e_data):
    output = {
        'scores': {},
        'labels': {},
    }
    output['labels'] = e_data['labels']
    for key in e_data['hyp'].keys():
        output['scores'][key] = 1 - similarity(e_data['hyp'][key], e_data['tgt'][key])

    return output

In [ ]:
sl_val_agnostic = calculate_similarity(e_val_agnostic)
sl_val_aware = calculate_similarity(e_val_aware)
sl_test = calculate_similarity(e_test)

# 5. Evaluation

In [ ]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats


def AUC_metric(scores, labels, plot=True):
    y = np.where(labels >= 0.5, 1, 0)
    pred = scores
    fpr, tpr, thresholds = metrics.roc_curve(y, pred)
    roc_auc = metrics.auc(fpr, tpr)
    display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                    estimator_name='example estimator')

    if plot:
        display.plot()
        plt.show()
    return roc_auc

def accuracy(scores, labels, threshold=0.5):
    y = np.where(labels >= 0.5, 1, 0)
    pred = np.where(scores >= threshold, 1, 0)

    return metrics.accuracy_score(y, pred)

def S_correlation(scores, labels):
    spearman_corr, p_value = scipy.stats.spearmanr(scores, labels)
    return spearman_corr

def evaluate(name, scores, labels, plot=True):
    print(name, "accuracy:", accuracy(scores, labels))
    print(name, "spearman correlation:", S_correlation(scores, labels))
    auc = AUC_metric(scores, labels, plot=plot)
    print(name, "auc:", auc)

In [ ]:
def evaluate_data(data):
    for task in tasks:
        print("task:", task)
        evaluate(f'Val Agnostic, task: {task}', data['scores'][task], data['labels'][task], False)

    print("All Data:")
    evaluate(f'Val Agnostic', data['scores']['all'], data['labels']['all'], True)

In [ ]:
evaluate_data(sl_val_agnostic)

In [ ]:
evaluate_data(sl_val_aware)

In [ ]:
evaluate_data(sl_test)

In [ ]:
# All test and val data together
all_scores = {}
all_labels = {}
all_sl = {}
for task in tasks + ['all']:
    all_scores[task] = np.concatenate([sl_val_agnostic['scores'][task], sl_val_aware['scores'][task], sl_test['scores'][task]])
    all_labels[task] = np.concatenate([sl_val_agnostic['labels'][task], sl_val_aware['labels'][task], sl_test['labels'][task]])

all_sl['scores'] = all_scores
all_sl['labels'] = all_labels

In [ ]:
evaluate_data(all_sl)

# 6. Results

Data         | AUC | Accuracy | Spearman Correlation |
----------   | --- | -------- | ---------------------|
Val Agnostic |0.73 | 0.65     | 0.47                 |
Val Aware    |0.69 | 0.66     | 0.39                 |
Test         |0.78 | 0.61     | 0.46                 |
All          |0.72 | 0.65     | 0.44                 |

<table>
    <tr>
        <th colspan="1">
            Data
        </th>
        <th colspan="4">
            AUC
        </th>
        <th colspan="4">
            Accuracy
        </th>
        <th colspan="4">
            Spearman Correlation
        </th>
    </tr>
    <tr>
        <th>
            ...
        </th>
        <th>
            MT
        </th>
        <th>
            DM
        </th>
        <th>
            PG
        </th>
        <th>
            All
        </th>
        <th>
            MT
        </th>
        <th>
            DM
        </th>
        <th>
            PG
        </th>
        <th>
            All
        </th>
        <th>
            MT
        </th>
        <th>
            DM
        </th>
        <th>
            PG
        </th>
        <th>
            All
        </th>
    </tr>
    <tr>
        <td>
            Val Agnostic
        </td>
        <td>
            0.87
        </td>
        <td>
            0.70
        </td>
        <td>
            0.67
        </td>
        <td>
            0.73
        </td>
        <td>
            62
        </td>
        <td>
            65
        </td>
        <td>
            71
        </td>
        <td>
            65
        </td>
        <td>
            0.61
        </td>
        <td>
            0.39
        </td>
        <td>
            0.46
        </td>
        <td>
            0.47
        </td>
    </tr>
    <tr>
        <td>
            Val Aware
        </td>
        <td>
            0.82
        </td>
        <td>
            0.72
        </td>
        <td>
            0.62
        </td>
        <td>
            0.69
        </td>
        <td>
            59
        </td>
        <td>
            65
        </td>
        <td>
            79
        </td>
        <td>
            66
        </td>
        <td>
            0.62
        </td>
        <td>
            0.48
        </td>
        <td>
            0.21
        </td>
        <td>
            0.39
        </td>
    </tr>
    <tr>
        <td>
            Test
        </td>
        <td>
            0.95
        </td>
        <td>
            0.71
        </td>
        <td>
            1
        </td>
        <td>
            0.78
        </td>
        <td>
            57
        </td>
        <td>
            61
        </td>
        <td>
            77
        </td>
        <td>
            61
        </td>
        <td>
            0.95
        </td>
        <td>
            0.32
        </td>
        <td>
            0.72
        </td>
        <td>
            0.46
        </td>
    </tr>
    <tr>
        <td>
            All Data
        </td>
        <td>
            0.85
        </td>
        <td>
            0.71
        </td>
        <td>
            0.67
        </td>
        <td>
            0.72
        </td>
        <td>
            60
        </td>
        <td>
            65
        </td>
        <td>
            75
        </td>
        <td>
            65
        </td>
        <td>
            0.64
        </td>
        <td>
            0.43
        </td>
        <td>
            0.37
        </td>
        <td>
            0.44
        </td>
    </tr>
</table>

# 7. Test On Samples

In [ ]:
def show_sample(data, task='all'):
    if task != 'all':
        sample = data[data['task'] == task].sample().iloc[0]
    else:
        sample = data.sample().iloc[0]

    hyp = sample['hyp']
    if sample['ref'] == 'either' or sample['ref'] == 'tgt':
        tgt = sample['tgt']
    else:
        tgt = sample['src']

    hyp, tgt = model.encode([hyp]), model.encode([tgt])
    score = 1 - similarity(hyp, tgt)

    print("Task:", task)
    print("Src:", sample['src'])
    print("Hyp:", sample['hyp'])
    print("Target:", sample['tgt'])
    print("p(Hullucinate):", sample['p(Hallucination)'])
    print("LaBSE p(Hullucinate):", score)
    print("")


In [ ]:
print("Val Agnostic\n")
for task in tasks + ['all']:
    show_sample(val_agnostic, task=task)

In [ ]:
print("Val Aware\n")
for task in tasks + ['all']:
    show_sample(val_aware, task=task)

In [ ]:
print("Test\n")
for task in tasks + ['all']:
    show_sample(test, task=task)